In [1]:
import os
import json
import numpy as np
from PIL import Image
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset_root = 'CSVD/rgb'
action_categories = ['club','grapple','highfive','idle','kick','slap']  # add all your categories

In [3]:
images = []
labels = []

In [4]:
# Function to load dataset
def load_dataset(root_path, categories):
    for category in categories:
        img_path = os.path.join(root_path, category, 'img')
        ann_path = os.path.join(root_path, category, 'ann')
        
        for filename in os.listdir(img_path):
            image_full_path = os.path.join(img_path, filename)
            annotation_full_path = os.path.join(ann_path, f'{filename}.json')
            
            # Load and process image
            with Image.open(image_full_path) as img:
                img = img.resize((224, 224))  # Resize to 224x224 or another size suitable for your CNN
                images.append(np.array(img))
                
            # Load and process annotation
            with open(annotation_full_path, 'r') as ann_file:
                ann_data = json.load(ann_file)
                # Here, we're assuming there's one main action per image
                # and we're using the first object's classTitle as the label
                labels.append(ann_data['objects'][0]['classTitle'])
    

    return np.array(images), labels

In [5]:
images, labels = load_dataset(dataset_root, action_categories)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)    

In [17]:
images = images / 255.0

In [18]:
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42
)

In [19]:
# Define CNN model
def create_cnn_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    
    # Create the CNN layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [20]:
input_shape = train_images[0].shape  # This should match the input size used in the `load_dataset` function
num_classes = len(np.unique(train_labels))
model = create_cnn_model(input_shape, num_classes)

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [22]:
# Train the model
history = model.fit(train_images, train_labels, batch_size=32, epochs=10,
                    validation_data=(val_images, val_labels))

Epoch 1/10
147/147 [==============================] - 51s 340ms/step - loss: 0.8167 - accuracy: 0.8283 - val_loss: 0.1787 - val_accuracy: 0.9718
Epoch 2/10
147/147 [==============================] - 46s 314ms/step - loss: 0.0828 - accuracy: 0.9778 - val_loss: 0.0743 - val_accuracy: 0.9812
Epoch 3/10
147/147 [==============================] - 48s 323ms/step - loss: 0.0456 - accuracy: 0.9883 - val_loss: 0.0531 - val_accuracy: 0.9838
Epoch 4/10
147/147 [==============================] - 47s 318ms/step - loss: 0.0198 - accuracy: 0.9951 - val_loss: 0.0544 - val_accuracy: 0.9863
Epoch 5/10
147/147 [==============================] - 46s 316ms/step - loss: 0.0238 - accuracy: 0.9947 - val_loss: 0.0299 - val_accuracy: 0.9932
Epoch 6/10
147/147 [==============================] - 47s 317ms/step - loss: 0.0161 - accuracy: 0.9962 - val_loss: 0.0440 - val_accuracy: 0.9906
Epoch 7/10
147/147 [==============================] - 47s 316ms/step - loss: 0.0140 - accuracy: 0.9966 - val_loss: 0.0431 - val_ac

In [23]:
from tensorflow.keras.preprocessing.image import img_to_array

# Function to preprocess a new image
def preprocess_image(image_path, target_size=(224, 224)):
    with Image.open(image_path) as img:
        img = img.resize(target_size)  # Resize it to match the input size of the model
        img = img_to_array(img)  # Convert the image to an array
        img = np.expand_dims(img, axis=0)  # Add a batch dimension
        img /= 255.0  # Normalize the image
    return img


In [25]:
test_image_path = 'CSVD/rgb/club/img/club-0002-frame1336_lXYkyNjb6L.jpg'
test_image = preprocess_image(test_image_path)

In [26]:
prediction = model.predict(test_image)

1/1 [==============================] - 0s 65ms/step


In [27]:
predicted_class_index = np.argmax(prediction, axis=1)
predicted_class = label_encoder.inverse_transform(predicted_class_index)  # Ensure label_encoder is the one used during training

In [29]:
predicted_class

array(['club'], dtype='<U10')

In [30]:
model.save('CSVD')

INFO:tensorflow:Assets written to: CSVD/assets


INFO:tensorflow:Assets written to: CSVD/assets


In [32]:
model.save('CSVD/your_model.h5')

/Users/asadullah/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
label_encoder.

array(['0-idle', '1-walk', '4-highfive', '6-pose', '7-slap', '9-grapple',
       'club', 'kick', 'pose'], dtype='<U10')

In [7]:
for index, label in enumerate(label_encoder.classes_):
    print(f"Index: {index}, Label: {label}")

Index: 0, Label: 0-idle
Index: 1, Label: 1-walk
Index: 2, Label: 4-highfive
Index: 3, Label: 6-pose
Index: 4, Label: 7-slap
Index: 5, Label: 9-grapple
Index: 6, Label: club
Index: 7, Label: kick
Index: 8, Label: pose


In [ ]:
['idle','walk','highfive','pose','slap','grapple','club','kick','pose']